In [2]:
! source /Users/miz/Downloads/bento_octo_milvus_RAG/env/bin/activate
! pip install bentoml sentence-transformers==2.2.2 sentencepiece==0.1.99 torch transformers==4.37.1 pymilvus octoai-sdk

Adding ocd to environment PATH
Adding homebrew to environment PATH
Setting up 'o-' aliases
Adding ocd to environment PATH
Adding homebrew to environment PATH
Setting up 'o-' aliases


Steps:
1. Spin up Bento Sentence Transformers Server. [Instructions here.](https://github.com/bentoml/BentoSentenceTransformers)
2. Embed data via Bento and store in Milvus via [Milvus Docker](https://milvus.io/docs/install_standalone-docker-compose.md)
3. Get LLM from [OctoAI](octoai.cloud)
4. Do RAG

Note: BentoSentenceTransformers already cloned into this repo

# Step 1: Spin up Bento Sentence Transformers Server. [Instructions here.](https://github.com/bentoml/BentoSentenceTransformers)

In [3]:
import bentoml

bento_client = bentoml.SyncHTTPClient("http://localhost:3000")

In [4]:
def get_embeddings(texts: list) -> list:
    if len(texts) > 25:
        splits = [texts[x:x+25] for x in range(0, len(texts), 25)]
        embeddings = []
        for split in splits:
            embedding_split = bento_client.encode(
                sentences = split
            )
            for embedding in embedding_split:
                embeddings.append(embedding)
        return embeddings
    return bento_client.encode(
        sentences=texts,
    )

# Step 2: Embed data via Bento and store in Milvus via [Milvus Docker](https://milvus.io/docs/install_standalone-docker-compose.md)

In [5]:
from pymilvus import connections, utility

In [26]:
COLLECTION_NAME = "bmo_test"
connections.connect(host="localhost", port=19530)

In [7]:
from pymilvus import FieldSchema, CollectionSchema, DataType

DIMENSION = 384

# id and embedding are required to define
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=DIMENSION)
]
# "enable_dynamic_field" lets us insert data with any metadata fields
schema = CollectionSchema(fields=fields, enable_dynamic_field=True)

In [8]:
from pymilvus import Collection

# define the collection name and pass the schema
collection = Collection(name=COLLECTION_NAME, schema=schema)

In [9]:
index_params = {
    "index_type": "HNSW", # one of 11 Milvus indexes
    "metric_type": "IP", # L2, Cosine, or IP
    "params": {
        "M": 8, # higher M = consumes more memory but better search quality
        "efConstruction": 64 # higher efConstruction = slower build, better search
    }, 
}

In [10]:
# pass the field to index on and the parameters to index with
collection.create_index(field_name="embedding", index_params=index_params)
# load the collection into memory
collection.load()

In [11]:
# naively chunk on newlines
def chunk_text(filename: str) -> list:
    with open(filename, "r") as f:
        text = f.read()
    sentences = text.split("\n")
    return sentences

In [12]:
import os

In [42]:
notes = os.listdir("data")

In [35]:
for note in notes:
    note = note.replace(" ", "")
    note = note.lower()
    print (note.lower())

mocharymethodcurriculum84f70e80ee9144f8afcada634936bfe9.md
morphontwitterthefutureofaicodeassistants03e7797b0d01429e9dd9b680c6550c69.md
recode,alightweightuserexperiencefortransfor5d4f5abf8d4a43f09a408376aefc2f89.md
redditdistributionfde4c8e76b3042f08905b492cd4aa030.md
guaranteedtokeepyouaccountablecommitactione0d5d396acc34265ba61c2ac0e271fbb.md
coolify1adba6fd61f5409aa51f71ada6ee99b5.md
gpt-neotwitterthreaddb3bb35435834bafb9baab45ceddea14.md
(46)8developerportfoliosthatmightbe1010's101690cfe0d745c3876e954fcc6138f1.md
g9automaticallyinteractivegraphics43b272493c9a415fa08b09f508f8ca82.md
showhnmathesar–open-sourcecollaborativeuifo1e90ec4a46c444748db8d45881b07d62.md
iinghatescience·buttondown3995c46a550c4ccdb71867be77bf19c4.md
pibox-nasraspberrypic4e4a4d2d5564a00b11832065a908631.md
ory-opensourceidentitysolutionsforeveryone7761439935c645b2a38ac8a407268592.md
sledchatwithyourdatawarehouseycombinator3adb9bc1982a4f16b5498022f153e3b4.md
grouparooopensourcedatasynchronizationframewo2e9a16d2fb

In [43]:
# store chunked text for each of the notes in a list of dicts
note_chunks = []
for note in notes:
    chunked = chunk_text(f"/Users/miz/Downloads/GoalGetter/data/{note}")
    cleaned = []
    for chunk in chunked:
        if len(chunk) > 7:
            cleaned.append(chunk)
    mapped = {
        "note_name": note.split(".")[0],
        "chunks": cleaned
    }
    note_chunks.append(mapped)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 3131: invalid start byte

In [21]:
print (notes)

['Mochary Method Curriculum 84f70e80ee9144f8afcada634936bfe9.md', 'Morph on Twitter The future of AI code assistants  03e7797b0d01429e9dd9b680c6550c69.md', 'reCode, a lightweight user experience for transfor 5d4f5abf8d4a43f09a408376aefc2f89.md', 'Reddit Distribution fde4c8e76b3042f08905b492cd4aa030.md', 'Guaranteed to keep you accountable Commit Action e0d5d396acc34265ba61c2ac0e271fbb.md', 'Coolify 1adba6fd61f5409aa51f71ada6ee99b5.md', 'GPT-Neo Twitter Thread db3bb35435834bafb9baab45ceddea14.md', "(46) 8 Developer Portfolios that MIGHT be 10 10's  101690cfe0d745c3876e954fcc6138f1.md", 'g9 Automatically Interactive Graphics 43b272493c9a415fa08b09f508f8ca82.md', 'Show HN Mathesar – open-source collaborative UI fo 1e90ec4a46c444748db8d45881b07d62.md', 'I ing hate Science · Buttondown 3995c46a550c4ccdb71867be77bf19c4.md', 'PiBox - NAS Raspberry Pi c4e4a4d2d5564a00b11832065a908631.md', 'Ory - Open Source Identity Solutions For Everyone  7761439935c645b2a38ac8a407268592.md', 'Sled Chat with 

In [ ]:
entries = []
for city_dict in city_chunks:
    embedding_list = get_embeddings(city_dict["chunks"]) # returns a list of lists
    # now match texts with embeddings and city name
    for i, embedding in enumerate(embedding_list):
        entry = {"embedding": embedding,
                 "sentence": city_dict["chunks"][i], # poorly named cuz it's really a bunch of sentences, but meh
                 "city": city_dict["city_name"]}
        entries.append(entry)

In [ ]:
collection.insert(entries)

In [ ]:
collection.flush()

# Step 3: Get LLM from [OctoAI](octoai.cloud)

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
os.environ["OCTOAI_TOKEN"] = os.getenv("OCTOAI_API_TOKEN")

In [ ]:
from octoai.client import Client

octo_client = Client()

# Step 4: Do RAG

In [ ]:
def dorag(question: str, context: str):

    completion = octo_client.chat.completions.create(
    messages=[
            {
                "role": "system",
                "content": f"You are a helpful assistant. The user has a question. Answer the user question based only on the context: {context}"
            },
            {
                "role": "user",
                "content": f"{question}"
            }
        ],
        model="nous-hermes-2-mixtral-8x7b-dpo",
        max_tokens=512,
        presence_penalty=0,
        temperature=0.1,
        top_p=0.9,
    )
    return completion.model_dump()

In [ ]:
def ask_a_question(question):
    embeddings = get_embeddings([question])
    res = collection.search(
        data=embeddings,  # search for the one (1) embedding returned as a list of lists
        anns_field="embedding",  # Search across embeddings
        param={"metric_type": "IP",
                "params": {"ef": 16}},
        limit = 5,  # get me the top 3 results
        output_fields=["sentence"]  # get the sentence/chunk and city
    )
    sentences = []
    for hits in res:
        for hit in hits:
            sentences.append(hit.entity.get("sentence"))
    context = ". ".join(sentences)
    return dorag(question, context)

In [ ]:
print(ask_a_question("What state is Cambridge in?")["choices"][0]["message"]["content"])

In [ ]:
# cleanup
# if utility.has_collection(COLLECTION_NAME):
#     utility.drop_collection(COLLECTION_NAME)